In [1]:
import cv2 as cv
import numpy as np
from tqdm.notebook import tqdm

## Camera calibration


In [2]:
class VideoReader(cv.VideoCapture):
    def __init__(self, folder, video, initial_frame=0):
        super().__init__(folder + video)
        self.actual_frame = initial_frame
        self.total_frames = self.get(cv.CAP_PROP_FRAME_COUNT)

    def get_actual_frame(self, shape=None, color=False):
        self.set(cv.CAP_PROP_POS_FRAMES, self.actual_frame)
        _, I = self.read()
        if shape:
            I = I[:, shape[0]:shape[1]]
        if color:
            I = cv.cvtColor(I, cv.COLOR_BGR2RGB)
        else:
            I = cv.cvtColor(I, cv.COLOR_BGR2GRAY)
        return I
    
    def get_frame_id(self):
        return self.actual_frame
    
    def get_total_frames(self):
        return self.total_frames
    
    def next_frame(self, plus_n=1):
        self.actual_frame += plus_n
    
    def is_finished(self):
        return not self.actual_frame < self.total_frames

In [3]:
# which video
FOLDER = "data/"
VIDEO = "calibration.mp4"

In [ ]:
# chess grid
grid = (9, 6)
# create object video
video = VideoReader(FOLDER, VIDEO, 0)
# number of frames to process
n_frames = 30
# calculate skip in order to have a different orientation of chess each time
skip = video.get_total_frames() // n_frames
condition_to_exit = True    
while condition_to_exit:
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    points_3d = np.zeros((np.prod(grid),3), np.float32)
    points_3d[:,:2] = np.mgrid[0:grid[0], 0:grid[1]].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    # 3d point in real world space
    real_3d_points_list = [] 
    # 2d points in image plane
    image_2d_points_list = [] 
    pbar = tqdm(total=video.total_frames)
    # find contours
    while not video.is_finished():
        # take frame
        I_g = video.get_actual_frame()
        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(I_g, grid, cv.CALIB_CB_ADAPTIVE_THRESH | cv.CALIB_CB_FAST_CHECK | cv.CALIB_CB_NORMALIZE_IMAGE)
        # If found, add points
        if ret == True:
            # add real 3d points
            real_3d_points_list.append(points_3d)
            # refine 2d points
            better_corners = cv.cornerSubPix(I_g, corners, (11,11), (-1,-1), (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            # add 2d points
            image_2d_points_list.append(better_corners)
        # change frame
        video.next_frame(skip)
        pbar.update(skip)
    pbar.close()
    # if there are sufficient data it exits from loop
    if len(image_2d_points_list) >= 20:
        condition_to_exit = False
    else:
        n_frames += 10
        # calculate skip in order to have a different orientation of chess each time
        skip = video.get_total_frames() // n_frames
        if skip < 1:
            raise ValueError("The algorithm isn't able to determine a sufficient number of corners")
    
video.release()
# calibrate camera
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(real_3d_points_list, image_2d_points_list, I_g.shape[::-1], None, None)
# save dictionary as file
np.save('intrinsic_parameters.npy', {K:mtx, distortion:dist})